### Stamps Extraction

In [1]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
from astropy.io import fits 
import copy
import subprocess as sp

PATH_TO_IMAGE = './CFIS/'
PATH_TO_STAMPS = './CFIS/Stamps/'

Running SExtractor on the FITS image

In [ ]:
FILE_NAME = PATH_TO_IMAGE + 'CFIS-000-0.fits'
FLAG_NAME = PATH_TO_IMAGE + 'CFIS_flag-000-0.fits'
WEIGHT_NAME = PATH_TO_IMAGE + 'CFIS_weight-000-0.fits'
CATALOG_NAME = PATH_TO_IMAGE + 'CFIS_cat-000-0.cat'

command = 'sex ' + FILE_NAME + ' -c cfis.sex -CATALOG_NAME ' + CATALOG_NAME + ' -WEIGHT_IMAGE ' + WEIGHT_NAME + ' -FLAG_IMAGE ' + FLAG_NAME
output = sp.Popen([command], stdout=sp.PIPE, stderr=sp.PIPE, shell=True).stderr.read()

In [ ]:
def correct(cat):
    
    # Open SExtractor catalog:
    cat = fits.open(cat)
    # Get the data from the second object of the fits file we opened
    data = cat[2].data
    cat.info()
    # Load the things needed into numpy arrays.
    imaflags = np.array([obj['IMAFLAGS_ISO']for obj in data])
    x = np.array([obj['X_IMAGE']for obj in data])
    Y = np.array([obj['Y_IMAGE']for obj in data])
    # To extract the objects interested
    ind = np.where(imaflags==0)
    data = data[ind[0]]
    flags = np.array([obj['FLAGS'] for obj in data])
    ind_flag = np.where(flags<=3)
    data = data[ind_flag[0]]
    return data

# Generate new data
data = correct(CATALOG_NAME)

fits.PrimaryHDU(data).writeto('')

# Features and labels after SExtractor
labels_sex = copy.deepcopy(data['FLAGS'])
feature_sex = copy.deepcopy(data['VIGNET'])
features_sex = copy.deepcopy(data['VIGNET'])
features_sex = features_sex.reshape(features_sex.shape[0],features_sex.shape[1]**2)

In [ ]:
def create_vig(data,check):
    '''Fonction to repair every object's vignet'''
    xs = np.array([obj['X_IMAGE'] for obj in data])
    ys = np.array([obj['Y_IMAGE'] for obj in data])
    flags = np.array([obj['FLAGS'] for obj in data])
    objs = np.array([obj['VIGNET'] for obj in data])

    M = fits.getdata(check)
    im = np.zeros(objs.shape)
    #print int(ys[1])-25:int(ys[1])+25
    for i in range(objs.shape[0]):
        #Adapt the indexes for different image size (here, for (116,116))    
        im[i] = M[int(ys[i])-58:int(ys[i])+58,int(xs[i])-58:int(xs[i])+58]
    return im

# Generate the full vignet for every object detected
data['VIGNET'] = create_vig(data_train,FILE_NAME)
# Getting sure the vignets have the right size
imgs = data['VIGNET'].reshape(116,116)

In [ ]:
def presentvig(data):

    '''Quick plot to check the source extraction'''

    objs = data['VIGNET']
    # Print some vignets of the object
    
    for j in np.random.choice(ind[0],30):
        plt.figure(figsize=(6, 6))  
        pic = objs[j]
        pic[pic<-1e29] = 0
        plt.imshow(pic, interpolation='Nearest', cmap='viridis')
        plt.colorbar()
    plt.show()     
    
presentvig(data)
